## fine-tuning

In [1]:
import json
import torch
import wandb
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    AutoConfig,
    BitsAndBytesConfig,
)

import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, TaskType
from tqdm import tqdm
tqdm.pandas()

/home/d-mishina/comments/attributes/lora/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Инициализация W&B
wandb.init(project="fine-tuning-lora", 
          )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dariamishina2812 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


### модель

In [3]:
quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )

In [4]:
model_name = "Vikhrmodels/Vikhr-7B-instruct_0.4" 

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto",
    quantization_config=quantization_config,
    # cache_dir='./models' #сильно замедляет загрузку
)

Loading checkpoint shards: 100%|██████████████████| 4/4 [00:05<00:00,  1.46s/it]


In [5]:
# Логирование имени модели
wandb.config.update({"model_name": model_name})

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
# В этой ячейке все веса изначальной модели замораживаются
for param in model.parameters():
  param.requires_grad = False  
  if param.ndim == 1:
    # в layernorm нужны очень маленькие числа, поэтому для него оставляют fp32 
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [8]:
# вспомогательная функция которая покажет сколько параметров будут обучаться
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

Адаптеры можно добавлять к полносвязным/линейным/dense слоям. В зависимости от архитектуры эти слои могут называться по-разному, поэтому их нужно указать вручную через парметр target_modules. Так как модели это в основном наслоенные однотипные трансформерные блоки, то перечислить нужно лишь несколько имен типовых слоев.

In [9]:
from peft import LoraConfig, get_peft_model
# Конфигурация LoRA:
# - task_type: для задач генерации текста используем CAUSAL_LM
# - r: ранг LoRA (обычно 8)
# - lora_alpha: коэффициент масштабирования
# - lora_dropout: dropout
# - target_modules: список модулей модели, к которым применяется LoRA.
config = LoraConfig(
    r=64, # внутренняя размерность адаптера, основной параметр
    target_modules=["q_proj", "k_proj", "v_proj", 'out_proj', 'fc1', 'fc2'], # к каким слоям добавлять адаптеры (подробнее выше)

    # "вес" адаптера, этот параметр делится на r, то есть если они равны то
    # вес адаптера = 1 (то есть базовая модель и адаптер одинаковы по значимости)
    # если поставить этот параметр выше, то адаптер будет сильнее влиять на базовую модель
    # как я понимаю никто особо не понимает что делать с этим параметром при обучении
    # лучше оставлять его равным r
    lora_alpha=128, 
    
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 37748736 || all params: 7665315840 || trainable%: 0.49246158655349026


In [10]:
# Логирование параметров LoraConfig
wandb.config.update({"lora_config": config.to_dict()})

In [11]:
model.print_trainable_parameters()

trainable params: 37,748,736 || all params: 7,665,315,840 || trainable%: 0.4925


### данные

In [12]:
#это то, что мы наразмечали чатгпт
data = pd.read_excel("prod_attr_10k.xlsx")
print(data.shape)
data.head()

(10387, 5)


,id_tov,name_tov,name_par_group,repr_text,attribute_descriptions
0,52,"Сметана 25%, 350 г",Молочная Продукция,Очень хорошая\nОчень хорошая сметана от Лебедя...,"{""id_tov"": 52, ""name_tov"": ""Сметана 25%, 350 г..."
1,61,"Молоко цельное, 1 л",Молочная Продукция,Молоко от Космол похоже на натуральное. Но не ...,"{""id_tov"": 61, ""name_tov"": ""Молоко цельное, 1 ..."
2,69,Запеканка классическая,Кулинария,Настоящий творог! Очень вкусно! Спасибо!\nДесе...,"{""id_tov"": 69, ""name_tov"": ""Запеканка классиче..."
3,75,"Простокваша 4%, 250 г",Молочная Продукция,Очень хз орошая простокваша! Густая и вкусная....,"{""id_tov"": 75, ""name_tov"": ""Простокваша 4%, 25..."
4,77,"Ряженка 4%, 450 г",Молочная Продукция,ряженка как ряженка\nЖидкая\nВкусная \nХорошая...,"{""id_tov"": 77, ""name_tov"": ""Ряженка 4%, 450 г""..."


In [13]:
data.attribute_descriptions[0]

'{"id_tov": 52, "name_tov": "Сметана 25%, 350 г", "name_par_group": "Молочная Продукция", "attribute_descriptions": [{"attribute": "вкус", "characteristic": "вкусная"}, {"attribute": "вкус", "characteristic": "вкус отличный"}, {"attribute": "вкус", "characteristic": "не вкусная"}, {"attribute": "вкус", "characteristic": "в меру кислая"}, {"attribute": "вкус", "characteristic": "кислая"}, {"attribute": "вкус", "characteristic": "жирная"}, {"attribute": "вкус", "characteristic": "вкуснейшая"}, {"attribute": "вкус", "characteristic": "вкуснющая"}, {"attribute": "густота", "characteristic": "густая"}, {"attribute": "густота", "characteristic": "плотная"}, {"attribute": "густота", "characteristic": "не достаточно густая"}, {"attribute": "консистенция", "characteristic": "гладкой"}, {"attribute": "кожа", "characteristic": "без кислятины"}, {"attribute": "свежесть", "characteristic": "свежая"}, {"attribute": "качество", "characteristic": "качество высокое"}]}'

In [14]:
# Функция для форматирования колонки attribute_descriptions.
# Она принимает значение из колонки (может быть строкой в формате JSON или уже словарём)
# и возвращает строку вида: "атрибут: характеристика; атрибут: характеристика; ..."
def format_attributes(attr_field):
    if isinstance(attr_field, str):
        try:
            attr_data = json.loads(attr_field)
        except Exception as e:
            print("Ошибка при разборе JSON:", e)
            attr_data = {}
    else:
        attr_data = attr_field
    if "attribute_descriptions" in attr_data:
        formatted = "; ".join([f"{item['attribute']}: {item['characteristic']}" 
                               for item in attr_data["attribute_descriptions"]])
    else:
        formatted = ""
    return formatted

In [15]:
format_attributes(data.attribute_descriptions[0])

'вкус: вкусная; вкус: вкус отличный; вкус: не вкусная; вкус: в меру кислая; вкус: кислая; вкус: жирная; вкус: вкуснейшая; вкус: вкуснющая; густота: густая; густота: плотная; густота: не достаточно густая; консистенция: гладкой; кожа: без кислятины; свежесть: свежая; качество: качество высокое'

In [16]:
dataset = Dataset.from_pandas(data[['repr_text',
       'attribute_descriptions']]) 

In [17]:
# # Функция для формирования обучающего примера.
# # В данном примере мы составляем инструкцию, где сначала идёт отзыв,
# # затем вопрос (инструкция), и после двоеточия — отформатированный ответ.
def generate_prompt(example):
    review = example["repr_text"]
    attributes = format_attributes(example["attribute_descriptions"])
    prompt = (
        "Ты помощник, который анализирует отзывы на товары. "
        "Ты получишь тексты отзывов покупателей об одном продукте. Выдели атрибуты товара (например, вкус, запах, текстура, цвет), про которые пишут покупатели."
        "У одного атрибута может быть много характеристик. Атрибут выражается 1-3 словами всегда, кратко и емко"
        "Чаще всего покупатели описывают характеристики атрибутов (например, сладкий, свежий, мягкий, сочный), а тебе нужно выделить именно сами атрибуты."
        "Ниже представлен список отзывов на мандарины, каждый новый отзыв с новой строки:"
        "Очень вкусные, сладкие! Никакие абхазские не нужны))"
        "Огромные, почти безвкусные"
        "Хорошие мандарины, в меру сладкие, без косточек"
        "Нам понравились! Очень свежие, прямо  на веточках и с листиками. Тонкая шкурка, чистится легко. На вкус разные. Зависит от спелости мандарина. Попадаются и кисленькие, и сладкие, и кисло-сладкие, но все сочные и нежные. Спасибо доставке по городу Одинцово!"
        "Вкусные свежие"
        "Мандарины без косточек, легко чистятся, в основном сладкие. По кислости попадаются разные, лично мне нравятся те, что покислее. Даже в одном мандарине иногда часть долек кислее других - странно. Иногда попадаются подсушенные мандарины, как будто их переморозили, но в основном мандарины хорошие."
        "Неплохие, но немного кисловаты."
        "сладкие, косточек не попалось, кожура тонкая и легко чистится"
        "Спасибо, тонкокорые,сочные ,сладкие,очень понравились"
        "Мандарины очень сладкие. Вкусные"
        "Любимые мандарины"
        "Безвкусные. Иран"
        "из этого списка отзывов выделяем следующие атрибуты и их характеристики:"
        "вкус: сладкий, безвкусный"
        "размер: огромный"
        "кожура: тонкая и легко чистится"
        "структура: без косточек, легко чистятся"
        "текстура: сочные, подсушенные"
        "Например, если пишут, что продукт кислый, сладкий, горький, это атрибут вкус, если пишут что продукт с комочками, волокнистый, то это атрибут консистенция." 
        "Если в отзыве упомянуто несколько атрибутов, извлекай их все. "
        "Возвращай результат в формате: список атрибутов и их описаний."
        f"Отзыв покупателя: {review}\n"
        f"Ожидаемый ответ: {attributes}\n"
    )
    return prompt

In [17]:
# тут просто промпт поменьше
# def generate_prompt(example):
#     review = example["repr_text"]
#     attributes = format_attributes(example["attribute_descriptions"])
#     prompt = (
        # " Ты помощник, который анализирует отзывы на товары. "
        # "Ты получишь тексты отзывов покупателей об одном продукте. Выдели атрибуты товара (например, вкус, запах, текстура, цвет), про которые пишут покупатели."
        # "У одного атрибута может быть много характеристик. Атрибут выражается 1-3 словами всегда, кратко и емко"
        # "Чаще всего покупатели описывают характеристики атрибутов (например, сладкий, свежий, мягкий, сочный), а тебе нужно выделить именно сами атрибуты."
        # "Например, если пишут, что продукт кислый, сладкий, горький, это атрибут вкус, если пишут что продукт с комочками, волокнистый, то это атрибут консистенция." 
        # "Если в отзыве упомянуто несколько атрибутов, извлекай их все. "
        # "Возвращай результат в формате: список атрибутов и их описаний."
#         f"Отзыв покупателя: {review}\n"
#         f"Ожидаемый ответ: {attributes}\n"
#     )
#     return prompt

In [18]:
dataset[0]

{'repr_text': 'Очень хорошая\nОчень хорошая сметана от Лебедяньмолоко. Спасибо.\nОтличная сметана\nХорошая сметана, очень нравится Пензенский комбинат, Н-Групп хуже по сравнению.\nГустая сметана, вкус отличный.\nВкусная! \nвкусная \nХорошая сметана  Из другого производства Мне понравилось Теперь буду запоминать ту производства 😊\nОчень вкусная, свежая сметана!\nГустая, ничего не скажешь, но вкуса натурального продукта НЕТ! Лучшая по вкусу Н- групп\nГУСТАЯ И ОЧЕНЬ ВКУСНАЯ СМЕТАНКА!\nВкуснейшая\nВкуснющая.\nВкусная пензенская сметана👍🏻\nОтличная \nВкусная, густая, не кислая!)\nВ прошлый раз понравилась (Гелиос), поэтому взяла без раздумий. \nВ этот - не достаточно густая, кисловатая, не вкусная. Но брэнд (Лебедяньмолоко) достойный, покупаю их рассыпчатый творог, сливочное масло 72.2% - без нареканий, поэтому убрала только одну звезду. И всё-таки теперь буду смотреть производителя...\nСметана АО"Гелиос" с ярким дрожжевым вкусом(\nСупер\nВкусная,плотная,в меру кислая. Брала для крема.\nСме

In [19]:
generate_prompt(dataset[0])

'Ты помощник, который анализирует отзывы на товары. Ты получишь тексты отзывов покупателей об одном продукте. Выдели атрибуты товара (например, вкус, запах, текстура, цвет), про которые пишут покупатели.У одного атрибута может быть много характеристик. Атрибут выражается 1-3 словами всегда, кратко и емкоЧаще всего покупатели описывают характеристики атрибутов (например, сладкий, свежий, мягкий, сочный), а тебе нужно выделить именно сами атрибуты.Ниже представлен список отзывов на мандарины, каждый новый отзыв с новой строки:Очень вкусные, сладкие! Никакие абхазские не нужны))Огромные, почти безвкусныеХорошие мандарины, в меру сладкие, без косточекНам понравились! Очень свежие, прямо  на веточках и с листиками. Тонкая шкурка, чистится легко. На вкус разные. Зависит от спелости мандарина. Попадаются и кисленькие, и сладкие, и кисло-сладкие, но все сочные и нежные. Спасибо доставке по городу Одинцово!Вкусные свежиеМандарины без косточек, легко чистятся, в основном сладкие. По кислости поп

In [20]:
# Применяем функцию generate_prompt ко всему датасету, создавая новую колонку "text"
dataset = dataset.map(lambda x: {"text": generate_prompt(x)})

Map: 100%|███████████████████████| 10387/10387 [00:01<00:00, 9087.31 examples/s]


In [21]:
def tokenize_function(example):
    return tokenizer(example["text"]
                     # , truncation=True, max_length=512
                    )

# Применяем токенизацию к датасету; убираем лишние колонки
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)

Map: 100%|███████████████████████| 10387/10387 [00:04<00:00, 2336.94 examples/s]


### дообучение

In [22]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=50, #100,
        max_steps=2000, #400,
        learning_rate=1e-4, #2e-4,
        fp16=True,
        logging_steps=10,
        report_to="wandb",
        output_dir='outputs' # папка для сохранения результатов
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False, )
)

In [23]:
# Логирование параметров TrainingArguments
wandb.config.update({"training_args": trainer.args.to_dict()})

In [24]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/home/d-mishina/comments/attributes/lora/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,2.983300
20,2.327400
30,1.875500
40,1.002200
50,1.214400
60,1.398400
70,1.503400
80,1.368300
90,1.148000
100,0.993700


/home/d-mishina/comments/attributes/lora/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/d-mishina/comments/attributes/lora/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/d-mishina/comments/attributes/lora/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=2000, training_loss=1.1498014419078826, metrics={'train_runtime': 11344.7798, 'train_samples_per_second': 0.705, 'train_steps_per_second': 0.176, 'total_flos': 3.525177901601587e+17, 'train_loss': 1.1498014419078826, 'epoch': 0.7701935111196688})

In [25]:
model.save_pretrained('Vikhr-7B-instruct_0.4_lora_r32_medium_prompt')

### проверяем 

In [1]:
# перед запуском этой ячейки нужно перезапустить кернел
import torch
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline


# peft_model_id = "Vikhr-7B-instruct_0.4_lora_r32" 
# peft_model_id = "Vikhr-7B-instruct_0.4_lora_r32_long_prompt" 
peft_model_id = "Vikhr-7B-instruct_0.4_lora_r32_medium_prompt" 
# peft_model_id = "Vikhr-7B-instruct_0.4_lora_r64_long_prompt" 
# peft_model_id = "Vikhr-7B-instruct_0.4_lora_r64_medium_prompt" 
# peft_model_id = "Vikhr-7B-instruct_0.4_lora_r64_long_prompt_50_2000_1e-4"


model_name = "Vikhrmodels/Vikhr-7B-instruct_0.4"

quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name,
                                             return_dict=True, 
                                             quantization_config=quantization_config,
                                             device_map='auto'
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/d-mishina/comments/attributes/lora/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:11<00:00,  2.87s/it]


In [2]:
config = dict(
    max_new_tokens=512, 
    do_sample=True, 
    num_beams=1, 
    temperature=0.25, 
    top_k=50, 
    top_p=0.98, 
    eos_token_id=79097
)

In [3]:
DEFAULT_SYSTEM_PROMPT  = ("Ты помощник, который анализирует отзывы на товары. "
        "Ты получишь тексты отзывов покупателей об одном продукте. Выдели атрибуты товара (например, вкус, запах, текстура, цвет), про которые пишут покупатели."
        "У одного атрибута может быть много характеристик. Атрибут выражается 1-3 словами всегда, кратко и емко"
        "Чаще всего покупатели описывают характеристики атрибутов (например, сладкий, свежий, мягкий, сочный), а тебе нужно выделить именно сами атрибуты."
        "Ниже представлен список отзывов на мандарины, каждый новый отзыв с новой строки:"
        "Очень вкусные, сладкие! Никакие абхазские не нужны))"
        "Огромные, почти безвкусные"
        "Хорошие мандарины, в меру сладкие, без косточек"
        "Нам понравились! Очень свежие, прямо  на веточках и с листиками. Тонкая шкурка, чистится легко. На вкус разные. Зависит от спелости мандарина. Попадаются и кисленькие, и сладкие, и кисло-сладкие, но все сочные и нежные. Спасибо доставке по городу Одинцово!"
        "Вкусные свежие"
        "Мандарины без косточек, легко чистятся, в основном сладкие. По кислости попадаются разные, лично мне нравятся те, что покислее. Даже в одном мандарине иногда часть долек кислее других - странно. Иногда попадаются подсушенные мандарины, как будто их переморозили, но в основном мандарины хорошие."
        "Неплохие, но немного кисловаты."
        "сладкие, косточек не попалось, кожура тонкая и легко чистится"
        "Спасибо, тонкокорые,сочные ,сладкие,очень понравились"
        "Мандарины очень сладкие. Вкусные"
        "Любимые мандарины"
        "Безвкусные. Иран"
        "из этого списка отзывов выделяем следующие атрибуты и их характеристики:"
        "вкус: сладкий, безвкусный"
        "размер: огромный"
        "кожура: тонкая и легко чистится"
        "структура: без косточек, легко чистятся"
        "текстура: сочные, подсушенные"
        "Например, если пишут, что продукт кислый, сладкий, горький, это атрибут вкус, если пишут что продукт с комочками, волокнистый, то это атрибут консистенция." 
        "Если в отзыве упомянуто несколько атрибутов, извлекай их все. "
        "Возвращай результат в формате: список атрибутов и их описаний.")

In [3]:
# DEFAULT_SYSTEM_PROMPT = (" Ты помощник, который анализирует отзывы на товары. "
#         "Ты получишь тексты отзывов покупателей об одном продукте. Выдели атрибуты товара (например, вкус, запах, текстура, цвет), про которые пишут покупатели."
#         "У одного атрибута может быть много характеристик. Атрибут выражается 1-3 словами всегда, кратко и емко"
#         "Чаще всего покупатели описывают характеристики атрибутов (например, сладкий, свежий, мягкий, сочный), а тебе нужно выделить именно сами атрибуты."
#         "Например, если пишут, что продукт кислый, сладкий, горький, это атрибут вкус, если пишут что продукт с комочками, волокнистый, то это атрибут консистенция." 
#         "Если в отзыве упомянуто несколько атрибутов, извлекай их все. "
#         "Возвращай результат в формате: список атрибутов и их описаний.")

In [4]:
# Загрузим LoRA веса 
model = PeftModel.from_pretrained(model, peft_model_id)

In [5]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'Jam

In [6]:
prompt = tokenizer.apply_chat_template([{
    "role": "system",
    "content": DEFAULT_SYSTEM_PROMPT
}, {
    "role": "user",
    "content": "Огромные и не очень вкусные Замечательные!! Мне очень понравились яблоки этого сорта. Сочные, сладкие! Вата, а не яблоки Сочные, вкусные Привезли очень крупные яблоки! На вкус ароматные и сладкие. Для тех кто любит мягкие. Сочные сладкие крепкие Яблоки просто восторг!- не ожидала! Сочные, ароматные, не жесткие- даже кожа хорошо жуётся! Сладкие. Брала со скидкой. Закажу ещё на завтра. Не уверена что привезут такие же- иначе бы кг 3 заказала. Привезли в заказе ужасные яблоки!!Есть такие невозможно (( Небольшие сочные хрустящие яблочки, блеск Крупненькие оказались, ожидала мелкие яблочки. Красивее, чем на фото.) Мне для пирога. Спасибо. Впервые в продаже такие красивые сезонные яблоки. И вкусные. Яблоки вкусные, но очень большие. Хотелось бы поменьше размером. Крупные. Мало косточек. Кисло сладкие. Мякоть мягкая и сочная. Кожица не толстая. Очень вкусные! Ужасные яблоки, кожура потемневшая, внутри ватные Очень вкусные и сладкие яблоки. Сочные. Приятные яблочки. Сочные, в меру сладкие. Кожица несколько плотновата. Хорошие крупные яблоки. Вкусные. Очень ароматные и сладкие! Те, что темно-красные даже очень сладкие! Хорошие мелкие сезонные яблоки. Адекватная цена. Вкусные яблоки, душистые"
}], tokenize=False, add_generation_prompt=True)
output = pipe(prompt, **config)
finetuned_output = output[0]['generated_text'][len(prompt):].strip()
finetuned_output

'Вкус: сладкий, сочный, ароматный, вкусный\nРазмер: крупные, средние, мелкие, очень большие\nКожица: тонкая, нежная, плотная, потемневшая\nВкус: кислый, кисло-сладкий, ватный, хрустящий\nСочность: сочные, хрустящие, мягкие\nВкус: вкусные, ароматные, сладкие, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вкусные, вку

In [7]:
attr = pd.DataFrame()
attr['repr_text'] = ["test", "Огромные и не очень вкусные Замечательные!! Мне очень понравились яблоки этого сорта. Сочные, сладкие! Вата, а не яблоки Сочные, вкусные Привезли очень крупные яблоки! На вкус ароматные и сладкие. Для тех кто любит мягкие. Сочные сладкие крепкие Яблоки просто восторг!- не ожидала! Сочные, ароматные, не жесткие- даже кожа хорошо жуётся! Сладкие. Брала со скидкой. Закажу ещё на завтра. Не уверена что привезут такие же- иначе бы кг 3 заказала. Привезли в заказе ужасные яблоки!!Есть такие невозможно (( Небольшие сочные хрустящие яблочки, блеск Крупненькие оказались, ожидала мелкие яблочки. Красивее, чем на фото.) Мне для пирога. Спасибо. Впервые в продаже такие красивые сезонные яблоки. И вкусные. Яблоки вкусные, но очень большие. Хотелось бы поменьше размером. Крупные. Мало косточек. Кисло сладкие. Мякоть мягкая и сочная. Кожица не толстая. Очень вкусные! Ужасные яблоки, кожура потемневшая, внутри ватные Очень вкусные и сладкие яблоки. Сочные. Приятные яблочки. Сочные, в меру сладкие. Кожица несколько плотновата. Хорошие крупные яблоки. Вкусные. Очень ароматные и сладкие! Те, что темно-красные даже очень сладкие! Хорошие мелкие сезонные яблоки. Адекватная цена. Вкусные яблоки, душистые"]
attr['prompt'] = ["test", DEFAULT_SYSTEM_PROMPT]
attr['ft_model'] = ["test", peft_model_id]
attr['ft_result'] = ["test", finetuned_output]

In [8]:
attr

,repr_text,prompt,ft_model,ft_result
0,test,test,test,test
1,Огромные и не очень вкусные Замечательные!! Мн...,Ты получишь тексты отзывов покупателей об одно...,Vikhr-7B-instruct_0.4_lora_r32_medium_prompt,"Вкус: сладкий, сочный, ароматный, вкусный\nРаз..."


In [9]:
final_attr = pd.DataFrame()
final_attr = final_attr._append(attr)
final_attr

,repr_text,prompt,ft_model,ft_result
0,test,test,test,test
1,Огромные и не очень вкусные Замечательные!! Мн...,Ты получишь тексты отзывов покупателей об одно...,Vikhr-7B-instruct_0.4_lora_r32,"Вкус: сладкий, ароматный, вкусный, кисло-сладк..."
2,test,test,test,test
3,Огромные и не очень вкусные Замечательные!! Мн...,"Ты помощник, который анализирует отзывы на тов...",Vikhr-7B-instruct_0.4_lora_r32,"Вкус: сладкий, безвкусный, кислый, кисло-сладк..."
4,test,test,test,test
5,Огромные и не очень вкусные Замечательные!! Мн...,"Ты помощник, который анализирует отзывы на то...",Vikhr-7B-instruct_0.4_lora_r32,"Вкус: сладкий, ароматный, кислый, ватный, сочн..."
6,test,test,test,test
7,Огромные и не очень вкусные Замечательные!! Мн...,"Ты помощник, который анализирует отзывы на тов...",Vikhr-7B-instruct_0.4_lora_r32_long_prompt,"Вкусные, сочные, сладкие, ароматные, крупные я..."
8,test,test,test,test
9,Огромные и не очень вкусные Замечательные!! Мн...,Ты получишь тексты отзывов покупателей об одно...,Vikhr-7B-instruct_0.4_lora_r32_long_prompt,"Вкус: вкусные, сладкие, ароматные, кисло-сладк..."


In [10]:
final_attr.to_excel('final_attr.xlsx', index=False)

In [43]:
wandb.log({"finetuned_output": finetuned_output})

In [26]:
wandb.finish()

train/epoch,▁▁▁▁▁▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇██
train/grad_norm,█▅▄▃▁▃▂▂▂▂▁▂▃▂▃▂ ▂▃▂▂▃▂▂▂▂▁▅▁▂▂▁▁▂▁▁▂▂▁▁
train/learning_rate,▁▂▂▄▄▄▄▅▇████▇▇▆▆▆▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁
train/loss,█▇▇▆▆▆▆▅▆▆▅▄▄▅▁▂▄█▆▅▅▄▅▅▆▆▅▆▃▅▆▅▅▅▅▆▅▅▆▆
total_flos,3.765620287625626e+16
train/epoch,0.15404
train/global_step,400
train/grad_norm,0.59094
train/learning_rate,0.0
train/loss,2.6251
